In [2]:
import yaml 
import requests
import pandas as pd
import sqlite3
from tqdm import tqdm
from bs4 import BeautifulSoup
from fpdf import FPDF
conn = sqlite3.connect("data/database.db")

In [3]:
tickers = pd.read_sql_query("SELECT * FROM master_ticker", conn)["Security"].drop_duplicates().tolist()
tickers = [t.replace(" ", "_") for t in tickers]

In [4]:
def create_pdf(text, filename="output.pdf"):
    pdf = FPDF()
    pdf.add_page()
    pdf.set_font("Arial", size=12)

    # Split text into lines that fit in the PDF
    lines = pdf.multi_cell(0, 10, text)

    pdf.output(filename)
    print(f"PDF saved as {filename}")

In [5]:
def extract_text_from_wikipedia(ticker):
    URL_BASE = "https://en.wikipedia.org/wiki/"
    request = requests.get(URL_BASE + ticker)
    soup = BeautifulSoup(request.text, "html.parser")
    #Extract text from article - Cleaning task and better scraping should be performed this is just for LLM read.
    pars = []
    for p in soup.find_all('p'):
        pars.append(str(p.text))
    
    text = "".join(pars).encode('latin-1', 'replace').decode('latin-1')
    create_pdf(text, filename="data/wikipedia/"+ str(ticker)+".pdf")

In [6]:
for t in tickers:
    extract_text_from_wikipedia(t)

UnicodeEncodeError: 'latin-1' codec can't encode character '\u2013' in position 399: ordinal not in range(256)